# Progetto di social computing 2025
### Analisi di dataset di scopus


## Analisi dataset


In [5]:
import pandas as pd
import networkx as nx
import json
from networkx.readwrite import json_graph
from itertools import combinations

# =========================
# Carica dataset
# =========================
df = pd.read_csv("../data/nuovo_dataset.csv")

# Rimuovi righe senza autori o anno
df = df.dropna(subset=['Author full names', 'Year'])

# =========================
# Crea grafi per anno
# =========================
grafi_per_anno = {}

for anno in df['Year'].unique():
    G = nx.Graph()
    df_anno = df[df['Year'] == anno]

    for authors_str in df_anno['Author full names']:
        authors = [a.strip() for a in authors_str.split(';')]
        # crea arco tra ogni coppia di autori
        for u, v in combinations(authors, 2):
            if G.has_edge(u, v):
                G[u][v]['weight'] += 1
            else:
                G.add_edge(u, v, weight=1)
    grafi_per_anno[anno] = G

print("Grafi creati per anno:", list(grafi_per_anno.keys()))

# =========================
# Dizionari per risultati
# =========================
central_node_per_anno = {}
highlight_nodes_per_anno = {}
subgrafi_per_anno = {}
pos_per_anno = {}

# =========================
# Nodo centrale, sottografi e serializzazione JSON
# =========================
for anno, G in grafi_per_anno.items():
    if len(G) == 0:
        continue

    # Nodo centrale (grado massimo)
    central = max(G.degree, key=lambda x: x[1])[0]
    central_node_per_anno[anno] = central

    # Vicini equicentrali
    central_degree = G.degree(central)
    equicentral_neighbors = [
        n for n in G.neighbors(central)
        if G.degree(n) == central_degree
    ]
    highlight_nodes_per_anno[anno] = [central] + equicentral_neighbors

    # Sottografo centrale + vicini
    neighbors = list(G.neighbors(central))
    nodes = [central] + neighbors
    H = G.subgraph(nodes).copy()
    subgrafi_per_anno[anno] = H

    # Posizioni
    initial_pos = {central: (0, 0)}
    pos = nx.spring_layout(H, seed=42, pos=initial_pos, fixed=[central])
    pos_per_anno[anno] = pos

    # Serializzazione JSON
    data_json = json_graph.node_link_data(H)
    filename = f"../graphs/subgrafo_{anno}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data_json, f, ensure_ascii=False, indent=2)

print("Serializzazione dei sottografi completata!")


FileNotFoundError: [Errno 2] No such file or directory: '../data/nuovo_dataset.csv'

In [ ]:
df.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'References', 'Correspondence Address', 'Publisher',
       'ISSN', 'Abbreviated Source Title', 'Document Type',
       'Publication Stage', 'EID'],
      dtype='object')

In [ ]:
grafi_unico_json = {}

for anno in df['Year'].unique():
    G = nx.Graph()
    df_anno = df[df['Year'] == anno]

    for authors_str in df_anno['Author full names']:
        authors = [a.strip() for a in authors_str.split(';')]
        for u, v in combinations(authors, 2):
            if G.has_edge(u, v):
                G[u][v]['weight'] += 1
            else:
                G.add_edge(u, v, weight=1)

    if len(G) > 0:
        central = max(G.degree, key=lambda x: x[1])[0]
        central_degree = G.degree(central)
        equicentral_neighbors = [
            n for n in G.neighbors(central)
            if G.degree(n) == central_degree
        ]
        nodes = [central] + equicentral_neighbors
        H = G.subgraph(nodes).copy()
    else:
        H = G

    # ✅ Chiave convertita in stringa
    grafi_unico_json[str(anno)] = json_graph.node_link_data(H)

# Salva in un unico JSON
with open("../graphs/grafi_tutti.json", "w", encoding="utf-8") as f:
    json.dump(grafi_unico_json, f, ensure_ascii=False, indent=2)

print("Tutti i sottografi salvati in un unico file: grafi_tutti.json")


Tutti i sottografi salvati in un unico file: grafi_tutti.json


In [ ]:
import json
from networkx.readwrite import json_graph
import networkx as nx

# Carica JSON
with open("../graphs/grafi_tutti.json", "r", encoding="utf-8") as f:
    dati = json.load(f)

# Esempio: ricostruire il grafo del 2020
G2020 = json_graph.node_link_graph(dati["2020"])
print(G2020.nodes(), G2020.edges())


['Gigli, Gian Luigi (7101781977)'] []


# INIZIO

LISTA AUTORI BELLA

In [ ]:
df["authors_list"] = (
    df["Author full names"]
    .str.split(";")
    .apply(lambda authors: [
        re.sub(r"\s*\([^)]*\)", "", a)  # rimuove (ID)
        .replace(",", "")               # rimuove virgole
        .strip()
        for a in authors
    ])
)
df["authors_list"]

NameError: name 'df' is not defined

Grafi per anno

In [ ]:
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
grafi_per_anno = {}
for anno, df_anno in df.groupby("Year"):
    G = nx.Graph()
    for authors in df_anno["authors_list"].dropna():
        for a1, a2 in itertools.combinations(authors, 2):
            if G.has_edge(a1, a2):
                G[a1][a2]["weight"] += 1
            else:
                G.add_edge(a1, a2, weight=1)
    grafi_per_anno[anno] = G

# GRAFI


GRAFO 1


In [6]:
# =========================
# Nodo centrale per anno
# =========================
central_node_per_anno = {}
for anno, G in grafi_per_anno.items():
    if len(G) == 0:
        continue
    central_node_per_anno[anno] = max(G.degree, key=lambda x: x[1])[0]

# =========================
# Vicini equicentrali del nodo centrale
# =========================
highlight_nodes_per_anno = {}
for anno, G in grafi_per_anno.items():
    if anno not in central_node_per_anno:
        continue
    central = central_node_per_anno[anno]
    central_degree = G.degree(central)
    equicentral_neighbors = [
        n for n in G.neighbors(central)
        if G.degree(n) == central_degree
    ]
    highlight_nodes_per_anno[anno] = [central] + equicentral_neighbors

# =========================
# Sottografo: centrale + vicini
# =========================
subgrafi_per_anno = {}
pos_per_anno = {}

for anno, G in grafi_per_anno.items():
    if anno not in central_node_per_anno:
        continue
    central = central_node_per_anno[anno]
    neighbors = list(G.neighbors(central))
    nodes = [central] + neighbors
    H = G.subgraph(nodes).copy()
    subgrafi_per_anno[anno] = H

    initial_pos = {central: (0, 0)}
    pos = nx.spring_layout(H, seed=42, pos=initial_pos, fixed=[central])
    pos_per_anno[anno] = pos

# =========================
# Funzione Plotly
# =========================
def plot_graph_plotly(G, pos, highlight_nodes):
    edge_x, edge_y = [], []
    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        mode="lines",
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        showlegend=False
    )

    node_x, node_y, texts, colors, sizes = [], [], [], [], []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        texts.append(f"{node}<br>Grado: {G.degree(node)}")

        if node in highlight_nodes:
            colors.append("crimson")
            sizes.append(11)
        else:
            colors.append("royalblue")
            sizes.append(8)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        text=texts,
        hoverinfo="text",
        marker=dict(
            size=sizes,
            color=colors,
            line=dict(width=0.5, color="black")
        ),
        showlegend=False
    )

    return edge_trace, node_trace

# =========================
# Genera titoli subplot
# =========================
authors_per_anno = {}
for anno, G in grafi_per_anno.items():
    if anno not in central_node_per_anno:
        continue
    central = central_node_per_anno[anno]
    central_degree = G.degree(central)
    equicentral_neighbors = [
        n for n in G.neighbors(central)
        if G.degree(n) == central_degree
    ]
    all_authors = [central] + equicentral_neighbors
    authors_per_anno[anno] = all_authors


anni = sorted(grafi_per_anno.keys())


titoli = []
for anno in anni:
    authors = authors_per_anno.get(anno, [])
    # massimo 3 autori
    if len(authors) > 3:
        authors = authors[:3] + ["..."]
    # raggruppa ogni 2 autori per riga
    lines = []
    for i in range(0, len(authors), 2):
        lines.append(", ".join(authors[i:i+2]))
    titolo = f"{anno}<br>" + "<br>".join(lines)
    titoli.append(titolo)

# =========================
# Crea figura 2x5
# =========================
fig = make_subplots(
    rows=2, cols=5,
    subplot_titles=titoli,
    horizontal_spacing=0.03,
    vertical_spacing=0.08
)

for i, anno in enumerate(anni):
    if anno not in subgrafi_per_anno:
        continue
    G = subgrafi_per_anno[anno]
    pos = pos_per_anno[anno]
    highlight_nodes = highlight_nodes_per_anno[anno]

    edge, node = plot_graph_plotly(G, pos, highlight_nodes)

    row = 1 if i < 5 else 2
    col = i + 1 if i < 5 else i - 4

    fig.add_trace(edge, row=row, col=col)
    fig.add_trace(node, row=row, col=col)

# =========================
# Layout finale
# =========================
fig.update_layout(
    height=650,
    width=1400,
    paper_bgcolor="white",
    plot_bgcolor="white",
    title_x=0.5,
    font=dict(size=10),
    margin=dict(l=20, r=20, t=50, b=20)
)

for r in [1, 2]:
    for c in range(1, 6):
        fig.update_xaxes(visible=False, row=r, col=c)  
        fig.update_yaxes(visible=False, row=r, col=c)  

fig.update_layout(
    margin=dict(t=50, b=20, l=20, r=20),
)

for ann in fig.layout.annotations:
    ann.font = dict(size=12)

fig.show()


NameError: name 'grafi_per_anno' is not defined

GRAFO 2

In [7]:
# ==============================
# Grafo totale e autore centrale globale
# ==============================
G_totale = nx.compose_all(grafi_per_anno.values())
central_author = max(G_totale.degree, key=lambda x: x[1])[0]
print("Autore centrale globale:", central_author)

# ==============================
# Creazione grafi cumulativi
# ==============================
grafi_cumulativi = {}
G_cumulativo = nx.Graph()

for anno in sorted(grafi_per_anno.keys()):
    G_cumulativo = nx.compose(G_cumulativo, grafi_per_anno[anno])
    grafi_cumulativi[anno] = G_cumulativo.copy()

# ==============================
# Calcolo layout per anno
# ==============================
pos_per_anno = {}
for anno, G in grafi_cumulativi.items():
    if central_author not in G:
        continue

    nodes_connected = nx.node_connected_component(G, central_author)
    G_sub = G.subgraph(nodes_connected).copy()

    initial_pos = {central_author: (0, 0)}

    pos = nx.spring_layout(
        G_sub,
        seed=42,
        pos=initial_pos,
        fixed=[central_author],
        k=0.5,
        iterations=200
    )

    xs = [x for x, y in pos.values()]
    ys = [y for x, y in pos.values()]
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)
    for node in pos:
        x, y = pos[node]
        norm_x = -1.1 + 2.2 * (x - min_x) / (max_x - min_x) if max_x > min_x else 0
        norm_y = -1.1 + 2.2 * (y - min_y) / (max_y - min_y) if max_y > min_y else 0
        pos[node] = (norm_x, norm_y)

    pos_per_anno[anno] = pos
    grafi_cumulativi[anno] = G_sub  

# ==============================
# Funzione di plotting
# ==============================
def plot_graph_plotly(G, pos, central_node):
    edge_x, edge_y = [], []

    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        mode="lines",
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        name="Collaborazioni",
        showlegend=True
    )

    node_x, node_y, texts, colors, sizes = [], [], [], [], []

    for node in G.nodes():
        if node == central_node:
            continue
        x, y = pos[node]
        deg = G.degree(node)
        node_x.append(x)
        node_y.append(y)
        texts.append(f"{node}<br>Collaborazioni: {deg}")
        colors.append("royalblue")
        sizes.append(8)

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers",
        hoverinfo="text",
        text=texts,
        marker=dict(
            size=sizes,
            color=colors,
            line=dict(width=0.5, color="black"),
            opacity=1.0
        ),
        name="Autori"
    )

    x_c, y_c = pos[central_node]
    node_central_trace = go.Scatter(
        x=[x_c],
        y=[y_c],
        mode="markers",
        hoverinfo="text",
        text=[f"{central_node}<br>Collaborazioni: {G.degree(central_node)}"],
        marker=dict(
            size=15,
            color="crimson",
            line=dict(width=0.5, color="black"),
            opacity=1.0
        ),
        name="Gigli G.L."
    )

    return edge_trace, node_trace, node_central_trace

# ==============================
# Creazione dei frame
# ==============================
frames = []

for anno in sorted(grafi_cumulativi.keys()):
    G = grafi_cumulativi[anno]
    if anno not in pos_per_anno: 
        continue
    pos = pos_per_anno[anno]
    edge, nodes, central = plot_graph_plotly(G, pos, central_author)
    frames.append(
        go.Frame(
            data=[edge, nodes, central],
            name=str(anno),
            layout=go.Layout(
                title=f"Collaborazioni cumulative fino al {anno}<br>Autore centrale: {central_author}"
            )
        )
    )


# ==============================
# Figura finale
# ==============================
fig = go.Figure(
    data=frames[0].data,
    frames=frames,
    layout=go.Layout(
        paper_bgcolor="white",
        plot_bgcolor="white",
        xaxis=dict(visible=False, range=[-1.5,1.5]),
        yaxis=dict(visible=False, range=[-1.5,1.5]),
        updatemenus=[{
            "type": "buttons",
            "buttons": [{
                "label": "Play",
                "method": "animate",
                "args": [None, {
                    "frame": {"duration":2000, "redraw":True},
                    "transition":{"duration":500},
                    "fromcurrent":True
                }]
            }]
        }],
        sliders=[{
            "steps": [{"method":"animate", "args":[[f.name]], "label":f.name} for f in frames]
        }]
    )
)

fig.show()


NameError: name 'grafi_per_anno' is not defined

GRAFO 3


In [12]:
import pandas as pd
from pyvis.network import Network
import json

def genera_grafo_statico(file_csv):
    try:
        # Caricamento del dataset fornito
        df = pd.read_csv(file_csv)
    except Exception as e:
        print(f"Errore nel caricamento del file: {e}")
        return

    # Calcolo frequenze e impatto citazioni dal dataset
    counts = df['Source title'].value_counts().to_dict()
    impact = df.groupby('Source title')['Cited by'].sum().to_dict() if 'Cited by' in df.columns else counts

    net = Network(height="900px", width="100%", bgcolor="#ffffff", font_color="black", select_menu=False)

    # Dizionario tematico tradotto e ottimizzato
    topic_mapping = {
        "Neuroscienze": {"color": "#9370db", "keywords": ["Brain", "Neuro", "Epileps", "Seizure", "Sleep", "Neurology", "Cerebellum", "Sclerosis"]},
        "Informatica e IA": {"color": "#97c2fc", "keywords": ["Computer", "Computing", "Software", "Logic", "Programming", "ACM", "Intelligence", "Automated Planning", "Heuristics"]},
        "Ingegneria e Segnali": {"color": "#40e0d0", "keywords": ["IEEE", "Signal", "Engineering", "Mechanical", "Electronics", "IMEKO", "Instruments", "Circuits", "Manufacturing", "Process Safety"]},
        "Scienze della Vita e Bio": {"color": "#ff69b4", "keywords": ["Bioinformatics", "Molecular", "Genetics", "Cancers", "Molecules", "Bio", "Nanoscale", "Thrombosis"]},
        "Matematica e Teoria": {"color": "#6a5acd", "keywords": ["Mathematics", "Matematica", "Algorithms", "Theory", "Calculus", "Formal", "Discrete", "Combinatorial", "Optimization"]},
        "Fisica": {"color": "#7be141", "keywords": ["Physics", "Nuclear", "Physical Review", "Frontiers in Physics"]},
        "Medicina Clinica": {"color": "#fb7e81", "keywords": ["Clinical", "Medicine", "Health", "Biomedica", "Acta Biomedica", "Expert Opinion"]},
        "Sostenibilità e Scienze Sociali": {"color": "#20b2aa", "keywords": ["Sustainability", "Education", "Science as Culture", "Transportation", "Social-Informatics"]},
        "Multidisciplinare": {"color": "#c0c0c0", "keywords": ["Scientific Reports", "Applied Sciences", "Nature", "MethodsX", "Proceedings of Science"]}
    }

    # Aggiunta Macro-aree (Dimensioni aumentate a 80)
    for topic, info in topic_mapping.items():
        net.add_node(topic, label=topic, color=info["color"], size=80, shape="ellipse")

    # Aggiunta Riviste dal dataset
    for title, freq in counts.items():
        if pd.isna(title): continue
        
        assigned_cats = []
        for topic, info in topic_mapping.items():
            for kw in info["keywords"]:
                if kw.lower() in str(title).lower():
                    assigned_cats.append(topic)
                    break
        
        if not assigned_cats: assigned_cats = ["Multidisciplinare"]

        primary_col = topic_mapping[assigned_cats[0]]["color"]
        imp_value = impact.get(title, 0)
        node_size = 15 + (min(imp_value, 50) / 2) if imp_value > 0 else 15
        
        net.add_node(title, label=title, shape="box", 
                     color={"border": primary_col, "background": "#ffffff"}, 
                     size=node_size)

        for cat in assigned_cats:
            net.add_edge(cat, title, value=freq, color=topic_mapping[cat]["color"])

    # CONFIGURAZIONE FISICA
    options = {
      "physics": {
        "forceAtlas2Based": {
          "gravitationalConstant": -150,
          "centralGravity": 0.01,
          "springLength": 200,
          "avoidOverlap": 1
        },
        "solver": "forceAtlas2Based",
        "stabilization": {
          "enabled": True,
          "iterations": 1000,
          "updateInterval": 25,
          "fit": True
        }
      },
      "nodes": {
        "borderWidth": 2
      },
      "interaction": {
        "dragNodes": True,
        "hover": True,
        "navigationButtons": True
      }
    }
    
    net.set_options(json.dumps(options))

    # Salvataggio file
    output_file = "grafo_statico.html"
    net.save_graph(output_file)

    # AGGIUNTA MANUALE DELLA LEGENDA HTML
    legenda_div = """
    <div style="position: fixed; top: 10px; right: 10px; width: 200px; padding: 10px; background-color: rgba(255,255,255,0.8); border: 1px solid #ccc; border-radius: 8px; z-index: 1000; font-family: Arial, sans-serif;">
        <h4 style="margin: 0 0 10px 0; font-size: 14px;">Legenda Aree</h4>
    """
    for topic, info in topic_mapping.items():
        legenda_div += f'<div style="display: flex; align-items: center; margin-bottom: 5px;"><div style="width: 12px; height: 12px; background-color: {info["color"]}; border-radius: 50%; margin-right: 8px;"></div><span style="font-size: 12px;">{topic}</span></div>'
    legenda_div += "</div>"

    with open(output_file, "r", encoding="utf-8") as f:
        html_content = f.read()

    # Inseriamo la legenda dopo il tag body
    html_content = html_content.replace('<body>', f'<body>{legenda_div}')

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(html_content)

    print("Grafo generato con successo")

# Esecuzione
genera_grafo_statico("nuovo_dataset.csv")

Grafo generato con successo


Analisi collaborazioni Udine - Italia - Estero

In [11]:
import pandas as pd
import re
from collections import Counter

# =====================================
# 1. CARICAMENTO E PULIZIA
# =====================================
df = pd.read_csv("nuovo_dataset.csv")
df = df.dropna(subset=["Affiliations"])

# 2. IDENTIFICAZIONE PAPER CON UDINE
df_udine = df[df["Affiliations"].str.contains("Udine", case=False, na=False)].copy()

# =====================================
# 3. PROCEDIMENTO DI CALCOLO
# =====================================
italy_counter = Counter()
foreign_counter = Counter()

# Fase A: Trova i nomi dei partner
for aff_string in df_udine["Affiliations"]:
    parts = [p.strip() for p in str(aff_string).split(";")]
    for p in parts:
        if "udine" in p.lower() or p == "": continue
        if "italy" in p.lower():
            italy_counter[p] += 1
        else:
            foreign_counter[p] += 1

# Fase B: Calcola i pesi (weight)
all_partners = list(italy_counter.keys()) + list(foreign_counter.keys())
partner_data = []

for p_name in all_partners:
    count = 0
    p_name_lower = p_name.lower()
    for row_aff in df_udine["Affiliations"]:
        if p_name_lower in str(row_aff).lower():
            count += 1
    
    if count >= 2: # Mantieni chi ha almeno 2 collaborazioni
        p_type = "Italian" if "italy" in p_name.lower() else "Foreign"
        partner_data.append({"Affiliation": p_name, "Type": p_type, "weight": count})

collab_counts = pd.DataFrame(partner_data)

# =====================================
# 4. FUNZIONE DI STAMPA (PULITA)
# =====================================
def print_left_aligned(df, title):
    print(f"\n{title}")
    if df.empty:
        print("Nessun dato trovato.")
        return
    df_str = df[['Affiliation', 'weight']].astype(str)
    widths = [max(df_str[col].str.len().max(), len(col)) for col in df_str.columns]
    header_row = "  ".join([col.ljust(widths[i]) for i, col in enumerate(df_str.columns)])
    print(header_row)
    print("-" * len(header_row))
    for _, row in df_str.iterrows():
        print("  ".join([row[col].ljust(widths[i]) for i, col in enumerate(df_str.columns)]))

# =====================================
# 5. OUTPUT FINALE (TOP 5)
# =====================================
top_italy = collab_counts[collab_counts["Type"]=="Italian"].sort_values(by="weight", ascending=False).head(5)
top_foreign = collab_counts[collab_counts["Type"]=="Foreign"].sort_values(by="weight", ascending=False).head(5)

print("CLASSIFICA TOP 5 COLLABORAZIONI TRA ITALIA ED ESTERO")
print_left_aligned(top_italy, "ITALIA:")
print_left_aligned(top_foreign, "ESTERO:")

CLASSIFICA TOP 5 COLLABORAZIONI TRA ITALIA ED ESTERO

ITALIA:
Affiliation                                                                                                weight
-----------------------------------------------------------------------------------------------------------------
Università degli Studi di Padova, Padua, PD, Italy                                                         8     
Università degli Studi di Trieste, Trieste, TS, Italy                                                      5     
Istituto Nazionale di Fisica Nucleare, Sezione di Trieste, Trieste, TS, Italy                              5     
Istituto Nazionale di Biostrutture e Biosistemi, Rome, RM, Italy                                           3     
Department of Medical Sciences and Public Health, Università degli Studi di Cagliari, Cagliari, CA, Italy  3     

ESTERO:
Affiliation                                                                                                                     wei